# Team 5 Mini Project - BADM 554

## Installing libraries

In [ ]:
!pip install sqlalchemy pyodbc pandas

In [ ]:
!pip install kagglehub

## Setting up connection to Azure SQL DB

In [18]:
from sqlalchemy import create_engine
import pandas as pd

# Azure SQL Database credentials
username = "gies"
password = "Password1"
server = "homework0.database.windows.net"
database = "Enterprise"

# Correct connection string (no :1433, no spaces in driver)
connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    "?driver=ODBC+Driver+18+for+SQL+Server"
    "&Encrypt=yes"
    "&TrustServerCertificate=no"
    "&LoginTimeout=60"
    "&ConnectionTimeout=60"
)

# Create engine with fast uploads
engine = create_engine(connection_string, fast_executemany=True, pool_pre_ping=True)


### Testing the connection

In [19]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("SELECT GETDATE();"))
    for row in result:
        print(row)

(datetime.datetime(2025, 11, 3, 19, 17, 33, 330000),)


### Loading libraries

In [4]:
import pandas as pd
import numpy as np
import json

In [5]:
import kagglehub
yelp_dataset_yelp_dataset_path = kagglehub.dataset_download('yelp-dataset/yelp-dataset')

print('Data source import complete.')

Data source import complete.


In [6]:
import os
for dirname, _, filenames in os.walk(yelp_dataset_yelp_dataset_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Users\ammra\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\Dataset_User_Agreement.pdf
C:\Users\ammra\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\yelp_academic_dataset_business.json
C:\Users\ammra\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\yelp_academic_dataset_checkin.json
C:\Users\ammra\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\yelp_academic_dataset_review.json
C:\Users\ammra\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\yelp_academic_dataset_tip.json
C:\Users\ammra\.cache\kagglehub\datasets\yelp-dataset\yelp-dataset\versions\4\yelp_academic_dataset_user.json


## Check in Dataset

In [ ]:
data_file_checkin = open(os.path.join(dirname, "yelp_academic_dataset_checkin.json"))
data_checkin = []
for line in data_file_checkin:
    data_checkin.append(json.loads(line))
checkin_df = pd.DataFrame(data_checkin)
data_file_checkin.close()

#### "Check-in for business" on Yelp is a user feature that allows a customer to mark their physical visit to a business through the Yelp app

In [ ]:
checkin_df.head()

In [ ]:
checkin_df.info()

#### Expanding the checkin_df dataset

In [ ]:
checkin_df1 = (
checkin_df.assign(date=checkin_df['date'].str.split(', ')).explode('date').reset_index(drop=True)
)
checkin_df1.head()

In [ ]:
## Dataset description
checkin_df1.describe()

1. Total number of rows after expanding the date field = 13,356,875
2. There are 131,930 unique businesses where users checked in at some point

In [ ]:
## Date Range:
print(checkin_df1['date'].max())
print(checkin_df1['date'].min())

 Dates are from 2009-12-30 to 2022-01-19

In [ ]:
## Top 10 businesses with highest check in counts
checkin_df1.groupby('business_id')['date'].agg(checkin_count = 'count').sort_values('checkin_count',ascending = False).head(10)

#### The primary key for checkin_df1 dataset would be a combination of business_id and check_in date


## Uploading to Azure SQL DB

In [ ]:
from sqlalchemy import create_engine

checkin_df1.to_sql("Check_in", con=engine, if_exists="replace", index=False, chunksize = 5000)

## Tip Dataset 

In [ ]:
data_file_tip = open(os.path.join(dirname, "yelp_academic_dataset_tip.json"), encoding="utf-8")
data_tip = []
for line in data_file_tip:
    data_tip.append(json.loads(line))
tip_df = pd.DataFrame(data_tip)
data_file_tip.close()

1. Tips are micro reviews given by users for each business. 
2. Compliment count is the number of compliments other users gave this review

In [ ]:
tip_df.head()

In [ ]:
tip_df.info()

In [ ]:
print(tip_df['user_id'].nunique())
print(tip_df['business_id'].nunique())
print(tip_df['text'].nunique())
print(tip_df['date'].nunique())
print(tip_df['compliment_count'].nunique())

##### This means:
1. One user can write multiple tips for different businesses.
2. One business can receive multiple tips from different users.
3. The same text can appear in multiple rows.

In [ ]:
## Date Range
print(tip_df['date'].max())
print(tip_df['date'].min())

### Dates are from 2009-04-16 to 2022-01-19

## Uploading to Azure SQL DB

In [ ]:
tip_df.to_sql("tip", con=engine, if_exists="replace", index=False,chunksize = 5000)

## BUSINESS DATASET

In [ ]:
data_file = open(os.path.join(dirname,"yelp_academic_dataset_business.json"),encoding="utf-8")
data = []
for line in data_file:
    data.append(json.loads(line))
business_df = pd.DataFrame(data)
data_file.close()

In [ ]:
business_df.head()

In [ ]:
business_df.info()

1. The dataset contains 150,346 businesses with 14 columns.
2. Some columns (attributes, categories, hours) have missing values.
3. Data types include text, numeric, and geographic fields.
4. Cleaning may be needed before analysis, especially expanding nested JSON fields in 'attributes' and 'hours'.

In [ ]:
## Missing values
business_df.isnull().sum()

In [ ]:
## Dataset description
business_df.describe()

1. The dataset contains 150,346 businesses with numeric data on location, rating, review counts, and status.
2. The average rating (stars) is ~3.6, with most businesses between 3.0 and 4.5 stars.
3. The average business has ~45 reviews, but counts range widely (up to 7,568).
4. About 80% of businesses are currently open.

In [ ]:
## Count of businesses per state
business_df['state'].value_counts().head(10)

1. The dataset includes businesses from multiple U.S. states and one Canadian province (AB = Alberta).
2. Pennsylvania (PA) has the largest number of businesses (~34K), followed by Florida (26K) and Tennessee (12K).

In [ ]:
## Count of businesses per city
business_df['city'].value_counts().head(10)

1. The dataset includes businesses from many cities, with Philadelphia leading (~14.5K businesses).
2. Tucson (9.2K) and Tampa (9.0K) follow, showing strong representation in the U.S. Southeast and Southwest.

In [ ]:
# Distribution of star ratings
business_df['stars'].value_counts().sort_index()

1. The most common rating is 4.0 stars (~31K businesses), followed by 4.5 and 3.5 stars.
2. Very few businesses have extremely low ratings (1.0–2.0), suggesting overall positive user experiences.
3. The distribution is right-skewed toward higher ratings, typical of review-based datasets.

In [ ]:
# Proportion of open vs closed businesses
business_df['is_open'].value_counts(normalize=True) * 100

About 79.6% of businesses in the dataset are currently open, while 20.4% are closed.

In [ ]:
# Businesses with most reviews
business_df[['name', 'city', 'stars', 'review_count']].sort_values('review_count', ascending=False).head(10)

In [ ]:
business_df['attributes'].head(5)

In [ ]:
# Extract all unique attribute keys
all_keys = set()

for attr in business_df['attributes'].dropna():
    if isinstance(attr, dict):
        all_keys.update(attr.keys())

print(f"Total unique keys: {len(all_keys)}")
print(sorted(all_keys)[:20])

1. The 'attributes' column contains 39 unique business characteristics.
2. These include operational features (e.g., 'DriveThru', 'ByAppointmentOnly'),amenities ('WiFi', 'BikeParking', 'BusinessParking'), and customer-related attributes ('GoodForKids', 'DogsAllowed', 'AcceptsInsurance').
####

In [ ]:
# Removing nested attributes from the business table
business_main = business_df.drop(columns=['attributes', 'hours', 'categories'])

In [ ]:
# Splitting categories column - Creating a separate categories table
categories_df = business_df[['business_id', 'categories']].copy()
categories_df = categories_df.dropna(subset=['categories'])
categories_df = categories_df.assign(
    category=categories_df['categories'].str.split(', ')
).explode('category')[['business_id', 'category']]

In [ ]:
# Flatten Attributes
from pandas import json_normalize
attributes_df = json_normalize(business_df['attributes']).add_prefix("attr_")
attributes_df.columns = [col.replace('.', '_') for col in attributes_df.columns]

In [ ]:
# Flatten Hours
hours_df = json_normalize(business_df['hours']).add_prefix("hours_")

In [ ]:
#Combining business main with attributes and hours
business_flat = pd.concat([business_main, attributes_df, hours_df], axis=1)

In [ ]:
business_flat.shape

In [ ]:
categories_df.shape

In [ ]:
business_main.shape

## Uploading to Azure SQL DB

In [ ]:
#uploading business flattened table
business_flat.to_sql('Business', engine, if_exists='replace', index=False, chunksize=500)

In [ ]:
#uploading business category table
categories_df.to_sql('BusinessCategory', engine, if_exists='replace', index=False, chunksize=500)

## User Dataset

In [7]:
data_file = open(os.path.join(dirname,"yelp_academic_dataset_user.json"),encoding="utf-8")
data = []
for line in data_file:
    data.append(json.loads(line))
user_df = pd.DataFrame(data)
data_file.close()

In [ ]:
user_df.head()

In [ ]:
# Column types, missing values
user_df.info()

In [ ]:
#Summary of numeric columns
user_df.describe()

In [ ]:
# List all fields
user_df.columns

In [ ]:
# Missing data per column
user_df.isnull().sum()

In [ ]:
#Finding the earliest time and latest time period of users
user_df['yelping_since'] = pd.to_datetime(user_df['yelping_since'])
user_df['yelping_since'].min(), user_df['yelping_since'].max()

1. The Yelp Academic Dataset (user file) contains user-level information including review counts, join dates, and social interactions (useful, funny, cool).
2. There are ~1.6 M users, with data ranging from 2004 to 2024.
3. Each record has 22 fields, mostly numeric or string, and a few lists (e.g., friends).

In [ ]:
user_df['friends'].head(3)
#The friends field contains a comma-separated list of user IDs representing the user’s social connections on Yelp.
#This is a nested string structure.

In [ ]:
user_df['elite'].head(3)
#This field lists the years when a user was designated as an “elite” member on Yelp.

In [ ]:
user_df['useful'].head(3)
#column is a numeric variable representing how many times a user’s reviews have been marked as useful by others.

In [8]:
# Removing nested attributes from the user table
user_main = user_df.drop(columns=['friends', 'elite'])

In [9]:
# Splitting the friends column - creating a separate friends table
friends_df = user_df[['user_id', 'friends']].dropna(subset=['friends'])
friends_df = friends_df.assign(friend_id=friends_df['friends'].str.split(', '))
friends_df = friends_df.explode('friend_id')[['user_id', 'friend_id']]

In [10]:
friends_df.shape

(106104025, 2)

In [11]:
# Splitting the elite column - creating a separate elite table
elite_df = user_df[['user_id', 'elite']].dropna(subset=['elite'])
elite_df = elite_df.assign(elite_year=elite_df['elite'].str.split(', '))
elite_df = elite_df.explode('elite_year')[['user_id', 'elite_year']]

In [12]:
elite_df.shape

(1987897, 2)

## Uploading to Azure SQL DB

In [ ]:
# Uploading user table
user_main.to_sql('UserProfile', engine, if_exists='replace', index=False, chunksize=500)

In [21]:
# Uploading friends table
friends_df.to_sql('UserFriends', engine, if_exists='replace', index=False, chunksize=20000)

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)

In [20]:
# Uploading elite table
elite_df.to_sql('UserElite', engine, if_exists='replace', index=False, chunksize=10000)

-199

## Review Dataset

In [ ]:

data = []
with open(os.path.join(dirname,"yelp_academic_dataset_review.json"),encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
        if len(data) >= 1000:
            break

review_df = pd.DataFrame(data)

In [ ]:
review_df.head()

In [ ]:
review_df.info()

In [ ]:
print(review_df.describe())

In [ ]:
cool_reviews = review_df[review_df['cool'] > 0]['text']
funny_reviews = review_df[review_df['funny'] > 0]['text']
cool_and_funny = review_df[(review_df['cool'] > 0) & (review_df['funny'] > 0)]['text']

In [ ]:
for text in cool_reviews[:5]: 
    print(text)
    print('-' * 50)

In [ ]:
for text in funny_reviews[:5]: 
    print(text)
    print('-' * 50)

In [ ]:
print(pd.to_datetime(review_df['date']).min())
print(pd.to_datetime(review_df['date']).max())
print(pd.to_datetime(review_df['date']).dt.year.unique())

In [ ]:
review_df.to_sql("review", con=engine, if_exists="replace", index=False,chunksize = 5000)